# Finding Text Bearing Surfaces

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.18.2`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:6.0.4`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [3]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}

## Load a CITE Library

In [4]:
//val filePath = s"https://raw.githubusercontent.com/homermultitext/hmt-archive/master/releases-cex/hmt-2020f.cex"
//val lib: CiteLibrary = CiteLibrarySource.fromUrl(filePath)

val filePath = s"hmt-test.cex"
val lib: CiteLibrary = CiteLibrarySource.fromFile(filePath)

Feb 10, 2020 11:11:44 AM wvlet.log.Logger log
INFO: Building text repo from cex ...
Feb 10, 2020 11:11:45 AM wvlet.log.Logger log
INFO: Building collection repo from cex ...
Feb 10, 2020 11:11:57 AM wvlet.log.Logger log
INFO: Building relations from cex ...
Feb 10, 2020 11:11:59 AM wvlet.log.Logger log
INFO: All library components built.


filePath: String = "hmt-test.cex"
lib: CiteLibrary = CiteLibrary(
  "Homer Multitext project, release 2020g",
  Cite2Urn("urn:cite2:hmt:publications.cex.2020g:all"),
  "Creative Commons Attribution, Non-Commercial 4.0 License <https://creativecommons.org/licenses/by-nc/4.0/>.",
  Vector(
    CiteNamespace("hmt", http://www.homermultitext.org/citens/hmt),
    CiteNamespace("greekLit", http://chs.harvard.edu/ctsns/greekLit)
  ),
  Some(
    TextRepository(
      Corpus(
        Vector(
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:1.1.lemma"),
            "\u03bc\u1fc6\u03bd\u03b9\u03bd \u1f04\u03b5\u03b9\u03b4\u03b5"
          ),
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:1.1.comment"),
            "\u03b6\u03b7\u03c4\u03bf\u1fe6\u03c3\u03b9 \u03b4\u03b9\u03b1 \u03c4\u03af \u1f00\u03c0\u1f78 \u03c4\u1fc6\u03c2 \u03bc\u03ae\u03bd\u03b9\u03b4\u03bf\u03c2 \u1f24\u03c1\u03be\u03b1\u03c4\u03bf \u03bf\u1f55\u03c4\u03c9\u03c2 \u03b

Get parts of the CITE Library in convenient form:

In [5]:
lazy val tr: TextRepository = lib.textRepository.get
lazy val corp: Corpus = tr.corpus
lazy val cat: Catalog = tr.catalog
lazy val colls: CiteCollectionRepository = lib.collectionRepository.get
lazy val rels: CiteRelationSet = lib.relationSet.get
lazy val myDseVec: DseVector = DseVector.fromCiteLibrary(lib)

tr: TextRepository = TextRepository(
  Corpus(
    Vector(
      CitableNode(
        CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:1.1.lemma"),
        "\u03bc\u1fc6\u03bd\u03b9\u03bd \u1f04\u03b5\u03b9\u03b4\u03b5"
      ),
      CitableNode(
        CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:1.1.comment"),
        "\u03b6\u03b7\u03c4\u03bf\u1fe6\u03c3\u03b9 \u03b4\u03b9\u03b1 \u03c4\u03af \u1f00\u03c0\u1f78 \u03c4\u1fc6\u03c2 \u03bc\u03ae\u03bd\u03b9\u03b4\u03bf\u03c2 \u1f24\u03c1\u03be\u03b1\u03c4\u03bf \u03bf\u1f55\u03c4\u03c9\u03c2 \u03b4\u03c5\u03c3\u03c6\u03ae\u03bc\u03bf\u03c5 \u1f40\u03bd\u1f79\u03bc\u03b1\u03c4\u03bf\u03c2\u00b7 \u03b4\u03b9\u03b1 \u03b4\u1f7b\u03bf \u03c4\u03b1\u1fe6\u03c4\u03b1\u00b7 \u03c0\u03c1\u1ff6\u03c4\u03bf\u03bd \u03bc\u1f72\u03bd \u1f35\u03bd' \u1f10\u03ba \u03c4\u03bf\u1fe6 \u03c0\u1f71\u03b8\u03bf\u03c5\u03c2 \u1f00\u03c0\u03ba\u03b1\u03c4\u03b1\u1fe5\u03c1\u03b5\u1f7b\u03c3\u1fc3 \u03c4\u1f78 \u03c4\u03bf\u03b9\u03bf\u1fe6\u03c4\u03bf \u03bc\u1f79\u03c1\u03b9\u03bf\u03bd \u03c4\u1fc6\u03c2 \u03c8\u03c5\u03c7\u1fc6\u03c2 \u03ba\u03b1\u1f76 \u03c0\u03c1\u03bf\u03c3\u03b5\u03ba\u03c4\u03b9\u03ba\u03c9\u03c4\u1f73\u03c1\u03bf\u03c5\u03c2 \u03c4\u03bf\u1f7a\u03c2 \u1f00\u03ba\u03c1\u03bf\u03b1\u03c4\u1f70\u03c2 \u1f10\u03c0\u03b9 \u03c4\u03bf\u1fe6 \u03bc\u03b5\u03b3\u1f73\u03b8\u03bf\u03c5\u03c2 \u03c0\u03bf\u03b9\u03ae\u03c3\u1fc3 \u03ba\u03b1\u1f76 \u03c0\u03c1\u03bf\u03c3\u03b5\u03b8\u03af\u03b6\u1fc3 \u03c6\u1f73\u03c1\u03b5\u03b9\u03bd \u03b3\u03b5\u03bd\u03bd\u03b1\u03af\u03c9\u03c2 \u1f21\u03bc\u1fb6\u03c2 \u03c4\u1f70 \u03c0\u1f71\u03b8\u03b7 . \u03bc\u1f73\u03bb\u03bb\u03c9\u03bd \u03c0\u03bf\u03bb\u03bb\u1f73\u03bc\u03bf\u03c5\u03c2 \u1f00\u03c0\u03b1\u03b3\u03b3\u1f73\u03bb\u03bb\u03b5\u03b9\u03bd\u00b7 \u03b4\u03b5\u1f7b\u03c4\u03b5\u03c1\u03bf\u03bd . \u1f35\u03bd\u03b1 \u03c4\u1f70 \u1f10\u03b3\u03ba\u1f7d\u03bc\u03b9\u03b1 \u03c4\u1ff6\u03bd \u1f19\u03bb\u03bb\u03ae\u03bd\u03c9\u03bd \u03c0\u03b9\u03b8\u03b1\u03bd\u1f7d\u03c4\u03b5\u03c1\u03b1 \u03c0\u03bf\u03b9\u03ae\u03c3\u1fc3\u00b7 \u1f10\u03c0\u03b5\u1f76 \u03b4\u1f72 \u1f14\u03bc\u03b5\u03bb\u03bb\u03b5 , \u03bd\u03b9\u03ba\u03c9\u03bd\u03c4\u03b1\u03c2 \u1f00\u03c0\u03bf\u03c6\u03b1\u03af\u03bd\u03b5\u03b9\u03bd \u03c4\u03bf\u1f7a\u03c2 \u1f19\u03bb\u03bb\u03b7\u03bd\u03b1\u03c2 , \u03b5\u1f30\u03ba\u1f79\u03c4\u03c9\u03c2 \u03bf\u1f50 \u03ba\u03b1\u03c4\u03b1\u03c4\u03c1\u1f73\u03c7\u03b5\u03b9 \u1f00\u03be\u03b9\u03bf\u03c0\u03b9\u03c3\u03c4\u1f79\u03c4\u03b5\u03c1\u03bf\u03bd \u1f10\u03ba \u03c4\u03bf\u1fe6 \u03bc\u1f74 \u03c0\u03b1\u03bd\u03c4\u03b1 \u03c7\u03b1\u03c1\u03af\u03b6\u03b5\u03c3\u03b8\u03b1\u03b9 \u03c4\u1ff7 \u03b5\u03ba\u03b5\u03af\u03...
corp: Corpus = [lazy]
cat: Catalog = Catalog(
  Vector(
    CatalogEntry(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:"),
      "book,scholion,part",
      "grc",
      "Iliadic scholia",
      "Main scholia of the Venetus A manuscript",
      Some("HMT project archival XML edition"),
      None,
      true
    ),
    CatalogEntry(
      CtsUrn("urn:cts:greekLit:tlg5026.msAext.hmt:"),
      "book,scholion,part",
      "grc",
      "Iliadic scholia",
      "Exterior scholia of the Venetus A manuscript",
      Some("HMT project archival XML edition"),
      None,
      true
    ),
    CatalogEntry(
      CtsUrn("urn:cts:greekLit:tlg5026.msAil.hmt:"),
      "book,scholion,part",
      "grc",
      "Iliadic scholia",
      "Interlinear scholia of the Venetus A manuscript",
      Some("HMT project archival XML edition"),
      None,
      true
    ),
    CatalogEntry(
      CtsUrn("urn:cts:greekLit:tlg5026.msAim.hmt:"),
      "book,scholion,part",
      "grc",
      "Iliadic scholia",
      "Intermarginal scholia of the Venetus A manuscript",
      Some("HMT project archival XML edition"),
...
colls: CiteCollectionRepository = [lazy]
rels: CiteRelationSet = [lazy]
myDseVec: DseVector = [lazy]

## Get a Catalog Entry for a Corpus

An example of working with the textual content of a CITE Library.

In [6]:
val su = CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:")
val entries = cat.entriesForUrn(su)

su: CtsUrn = CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:")
entries: Vector[CatalogEntry] = Vector(
  CatalogEntry(
    CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:"),
    "book,scholion,part",
    "grc",
    "Iliadic scholia",
    "Main scholia of the Venetus A manuscript",
    Some("HMT project archival XML edition"),
    None,
    true
  )
)

In [7]:
val newCat = Catalog(entries)

newCat: Catalog = Catalog(
  Vector(
    CatalogEntry(
      CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:"),
      "book,scholion,part",
      "grc",
      "Iliadic scholia",
      "Main scholia of the Venetus A manuscript",
      Some("HMT project archival XML edition"),
      None,
      true
    )
  )
)

In [8]:
newCat.cex("#")


res7: String = """urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang
urn:cts:greekLit:tlg5026.msA.hmt:#book,scholion,part#Iliadic scholia#Main scholia of the Venetus A manuscript#HMT project archival XML edition##true#grc
"""